In [1]:
import numpy as np
from jina import DocumentArray, Executor, requests

In [ ]:
import nest_asyncio
nest_asyncio.apply()
__import__('IPython').embed()

Python 3.9.7 (default, Sep 16 2021, 13:09:58) 
Type 'copyright', 'credits' or 'license' for more information
IPython 7.29.0 -- An enhanced Interactive Python. Type '?' for help.

In [1]: q
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<ipython-input-1-e222fcdf3350> in <module>
----> 1 q

NameError: name 'q' is not defined



In [2]:
import pandas as pd
data = pd.read_csv('SAP-Data.csv')

In [3]:
data['text'] = data[['SAP_Part_Short','Desc(40)','Long Text']].apply(lambda x: str(x[1])+' '+str(x[2]),axis=1)

In [4]:
from collections import defaultdict
li = data[['SAP_Part_Short','text']].values.tolist()
d = defaultdict(list)
for item in li:
    d[item[1].lower()].append(item[0])

In [5]:
to_index = [from collections import defaultdict
li = data[['SAP_Part_Short','text']].values.tolist()
d = defaultdict(list)
for item in li:
    d[item[1].lower()].append(item[0])item[0] for item in d.items()]
len(to_index),len(li)

(400885, 406847)

In [6]:
import joblib
joblib.dump(d,'SAP_data.pkl')

['SAP_data.pkl']

In [8]:
import numpy as np
from jina import DocumentArray, Executor, requests


class CharEmbed(Executor):  # a simple character embedding with mean-pooling
    offset = 32  # letter `a`
    dim = 127 - offset + 1  # last pos reserved for `UNK`
    char_embd = np.eye(dim) * 1  # one-hot embedding for all chars

    @requests
    def foo(self, docs: DocumentArray, **kwargs):
        count=0
        for d in docs:
            #print (d)
            r_emb = [ord(c) - self.offset if self.offset <= ord(c) <= 127 else (self.dim - 1) for c in d.text]
            #print (r_emb)
            d.embedding = self.char_embd[r_emb, :].mean(axis=0)  # average pooling
            count+=1
            if count%10000==0: print (count)

In [9]:

class Indexer(Executor):
    _docs = DocumentArray()  # for storing all documents in memory

    @requests(on='/index')
    def foo(self, docs: DocumentArray, **kwargs):
        self._docs.extend(docs)  # extend stored `docs`

    @requests(on='/search')
    def bar(self, docs: DocumentArray, **kwargs):
        docs.match(self._docs, metric='euclidean', limit=10)


In [14]:
from jina import Flow

f = (Flow(port_expose=12345, protocol='http', cors=True)
        .add(uses=CharEmbed, replicas=5)
        .add(uses=Indexer))  # build a Flow, with 2 shard CharEmbed, tho unnecessary
f.plot('flow.svg')

In [ ]:
from jina import Document

with f:
    #f.post('/index', (Document(text=t.strip()) for t in open('test.txt') if t.strip()))  # index all lines of _this_ file
    f.post('/index', (Document(text=t.strip()) for t in to_index if t.strip()))  # index all lines of _this_ file
    
    f.block()  # block for listening request

In [ ]:
[item for item in to_index if len(item)<10]